In [8]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from scipy.stats import invgamma
from sklearn.metrics import f1_score
from sklearn.preprocessing import scale

In [9]:
df = pd.read_csv('creditcard.csv')

In [10]:
y = df["Class"].values
df2 = df.drop("Class", axis=1).values
df2 = scale( df2, axis=0, with_mean=True, with_std=True, copy=True )

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(df2, y, test_size = 0.2, random_state=1)

In [12]:
clf = GaussianNB()
y_pred = clf.fit(xtrain, ytrain).predict(xtest)
print("Correct classification", sum(y_pred==ytest)/len(ytest)*100, "%")

Correct classification 97.8266212563 %


In [13]:
print("F1 score: ", f1_score(ytest, y_pred,average='binary'))

F1 score:  0.0950292397661


In [ ]:
# with tau = 1
Y = ytrain
X=xtrain
Z=Y
K = len(X[0,:])
N = len(Y)
# inverse of prior variance for beta
B0 = np.identity(K)*.001
xi =  np.mean(X, axis = 0)
XX = np.matmul(np.transpose(X),X)
XX_inv = np.linalg.inv(XX)
pos = Y>0
neg = Y<=0
no_pos = sum(pos)
no_neg = sum(neg)
nrep = 10000
tau = 1
sig = np.linalg.inv(XX+B0)
beta = np.zeros((nrep,K))
# Gibb sampling
for i in range(1,nrep):
    b_mean = np.matmul(sig, tau*np.matmul(np.transpose(X),Z))
    beta[i,:] = np.random.multivariate_normal(b_mean, sig)
    muy = np.matmul(X,np.transpose(beta[i,:]))
    # A and B are just some operations to simplify the operations pf Z
    A = np.ones(no_pos) - norm.cdf(-muy[pos])
    B = np.random.uniform(0,1,no_pos)
    Z[pos] = norm.ppf(np.multiply(A,B) +norm.cdf(-muy[pos])) +muy[pos]
    A = np.random.uniform(0,1,no_neg)
    B = norm.cdf(-muy[neg])
    Z[neg] = norm.ppf(np.multiply(A,B))+muy[neg]

b = np.mean(beta[1000:nrep,:], axis = 0)
z_pred = np.matmul(xtest,b)
y_pred = z_pred>0
print("Correct classification", f1_score(ytest, y_pred,average='binary')*100, "%")

In [ ]:
print("F1 score: ", f1_score(ytest, y_pred,average='binary'))